In [49]:
from osgeo import gdal
import fiona
import numpy as np
import pandas as pd
import geopandas as gpd

import rasterio as rio
#rom rasterstats import zonal_stats

from nansat import Nansat
from nansat import *

from s1denoise import Sentinel1Image
from s1denoise import *

import matplotlib.pyplot as plt
%matplotlib inline

import scipy
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

import matplotlib as mpl

In [13]:
s1_path = '/container_dir1/subsets/S1B_IW_GRDH_1SDV_20210716T054037_20210716T054102_027815_0351B0_E546.zip'
s1 = Nansat(s1_path)

s2_path = '/container_dir1/subsets/Subset_S2_20210716_resampled.tif'
src_awei = '/container_dir1/subsets/AWEI_S2B_MSIL2A_20210716.tif'
src_ndwi = '/container_dir1/subsets/NDWI_S2B_MSIL2A_20210716.tif'
src_vis = '/container_dir1/subsets/VIs_Subset_S2B_MSIL2A_20210716.tif'

12:17:45|40|nansat|_get_dataset_metadata|GDAL could not open /container_dir1/subsets/S1B_IW_GRDH_1SDV_20210716T054037_20210716T054102_027815_0351B0_E546.zip, trying to read with Nansat mappers...


In [14]:
ndwi = Nansat(src_ndwi)
awei = Nansat(src_awei)
raw_bands_s2 = Nansat(s2_path)
vis = Nansat(src_vis)

In [15]:
#Reproject the S1 data to S2
n = gdal.Open(s2_path)
d = Domain(ds=n)
s1.reproject(d)

In [16]:
# Stack as multi-dimensional np array
all_bands = [raw_bands_s2[band_n] for band_n in raw_bands_s2.bands().keys()]
all_bands.append(ndwi[1])
all_bands.append(awei[1])
for band_n in vis.bands().keys():
    all_bands.append(vis[band_n])
lon_grd, lat_grd = ndwi.get_geolocation_grids()

In [18]:
all_bands.append(s1[8])
all_bands.append(s1[10])

In [21]:
bands_stack = np.array(all_bands)
bands_stack.shape

(15, 1091, 960)

In [ ]:
# Bloody export this raster stack


In [55]:
# Mask the array/ rasterbandstack
aoi_path = '/container_dir1/subsets/aoi.shp'
mask = gpd.read_file(aoi_path)

In [56]:
mask_arr = np.array(mask)

In [57]:
landmask = np.ma.masked_array(bands_stack, mask_arr)

MaskError: Mask and data not compatible: data size is 15710400, mask size is 2.

In [47]:
#Extract the pixel values of VIs & Bands at training points
src_t_points = '/container_dir1/subsets/t_points_new2.shp'
points = gpd.read_file(src_t_points)
class_names = points['classtxt'].unique()
print(class_names)
# create new working gdf with only the interesting columns and without the qgis stuff in original table
# points2 = points[['classid', 'classtxt', 'geometry', 'x', 'y']].copy()
points2 = points[['classid', 'classtxt', 'geometry']].copy()
# Rename x and y to lon and lat
# points2.rename(columns={'x':'lon', 'y':'lat'}, inplace=True)
points2

['water' 'structures' 'seabed' 'lsm' 'pioneer' 'usm']


,classid,classtxt,geometry
0,1,water,POINT (486073.904 6046397.927)
1,1,water,POINT (486080.623 6046405.312)
2,1,water,POINT (486087.343 6046412.697)
3,1,water,POINT (486094.062 6046420.083)
4,1,water,POINT (486100.782 6046427.468)
...,...,...,...
14140,5,lsm,POINT (489238.936 6051398.693)
14141,5,lsm,POINT (489231.804 6051405.338)
14142,5,lsm,POINT (489224.672 6051411.984)
14143,5,lsm,POINT (489217.540 6051418.630)


In [61]:
### Pixel values with geopandas
coord_list = [(x,y) for x,y in zip(points2['geometry'].x , points2['geometry'].y)]

#list_tifs =  [src_ndwi, src_awei, src_bands, src_vis]
# names = [rio_ndwi, rio_awei, rio_bands, rio_vis]
#for i in len(list_tifs):
#    with rio.open(list_tifs[i]) as names(i)

## when opening stacked tif:
#open raster, call sample values
#with rasterio.open(fp_raster,'r') as src: 
#    samplevalues = [x for x in src.sample(coord_list)]
#bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']

rio_ndwi = rio.open(src_ndwi)
rio_awei = rio.open(src_awei)
rio_bands = rio.open(s2_path)
rio_vis = rio.open(src_vis)

points2['ndwi'] = [x for x in rio_ndwi.sample(coord_list)]
points2['awei'] = [x for x in rio_awei.sample(coord_list)]
points2['band values'] = [x for x in rio_bands.sample(coord_list)]
points2['vis'] = [x for x in rio_vis.sample(coord_list)]
#gdf['B4'] = [x for x in rio_bands[3].sample(coord_list)]
#gdf['B8'] = [x for x in rio_bands[4].sample(coord_list)]
#gdf['B11'] = [x for x in rio_bands[5].sample(coord_list)]
#gdf['B12'] = [x for x in rio_bands[6].sample(coord_list)]
#gdf['NDVI'] = [x for x in rio_vis[1].sample(coord_list)]
#gdf['SAVI'] = [x for x in rio_vis[2].sample(coord_list)]
#gdf['MSAVI'] = [x for x in rio_vis[3].sample(coord_list)]
#gdf['NDMI'] = [x for x in rio_vis[4].sample(coord_list)]
#gdf['EVI'] = [x for x in rio_vis[5].sample(coord_list)]

#points2.drop(columns=['lon','lat_x','lat_y'])
points2.head()

,classid,classtxt,geometry,ndwi,awei,band values,vis
0,1,water,POINT (486073.904 6046397.927),[0.44288576],[0.035224993],"[248, 360, 200, 139, 167, 140]","[-0.179941, 2.7980895, 0.022683946, -0.0915032..."
1,1,water,POINT (486080.623 6046405.312),[0.44268772],[0.0382],"[255, 365, 225, 141, 161, 145]","[-0.2295082, 2.7797241, 0.022445843, -0.066225..."
2,1,water,POINT (486087.343 6046412.697),[0.3942857],[0.038150005],"[236, 366, 248, 159, 161, 145]","[-0.2186732, 2.7577214, 0.025438169, -0.006249..."
3,1,water,POINT (486094.062 6046420.083),[0.48669198],[0.046324994],"[203, 391, 240, 135, 165, 148]","[-0.28, 2.7711627, 0.0208721, -0.09999999, -0...."
4,1,water,POINT (486100.782 6046427.468),[0.4201835],[0.04925],"[257, 387, 238, 158, 155, 144]","[-0.20202024, 2.765011, 0.025494339, 0.0095846..."


In [65]:
### Export geodataframe as trainingpoints for classification
points2.to_file('/container_dir1/results/gdf_train_pixelvalues.shp', driver='CSV')
# doesn´t work because something is a np array?

ValueError: Invalid field type <class 'numpy.ndarray'>

In [67]:
points2.columns

Index(['classid', 'classtxt', 'geometry', 'ndwi', 'awei', 'band values',
       'vis'],
      dtype='object')

In [66]:
# Start splitting training points into train and test data set
tpoints_path = '/container_dir1/results/gdf_train_pixelvalues.shp'
# tpoints = gdp.open(tpoints_path)
cols = ['ndwi', 'awei', 'band values', 'vis'] # 'NDVI', 'SAVI', 'MSAVI', 'NDMI', 'EVI', 'AWEI', 'NDWI']

### Splitting the training points into training and testing data for the RandomForest Model
X_train, X_test, y_train, y_test=train_test_split(points2[cols], points2['classid'], train_size=0.7, random_state=42)
X_train
print(y_train)

4053     3
12242    3
108      1
6264     3
9552     6
        ..
5191     3
13418    4
5390     3
860      1
7270     3
Name: classid, Length: 9901, dtype: int64


In [54]:
### Normal classification without scaling
clf = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state=50)
clf.fit(X_train, y_train)

4053     3
12242    3
108      1
6264     3
9552     6
        ..
5191     3
13418    4
5390     3
860      1
7270     3
Name: classid, Length: 9901, dtype: int64


ValueError: setting an array element with a sequence.

In [45]:
rio_bands.shape

(1091, 960)